# 03-Calculate Bearings

This notebook estimates the bearings (azimuth) for all locations in the vehicle trips. The bearing is the direction of movement measured in decimal degrees, with zero facing north and angles increasing in the clockwise direction. 

**Requirements:**

- Please run the `02-import-data.ipynb` notebook first and its dependencies.
- Recommended install: [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/user_install.html). Enable using `jupyter nbextension enable --py widgetsnbextension --sys-prefix` for Jupyter Notebook and `jupyter labextension install @jupyter-widgets/jupyterlab-manager` for Jupyter Lab.

In [1]:
import os
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from db.api import EVedDb
from geo.math import vec_bearings

In [2]:
db = EVedDb()

In [3]:
trips = db.query("select distinct vehicle_id, trip_id from signal;")

In [4]:
len(trips)

32552

In [5]:
def get_trip_locations(vehicle_id, trip_id):
    sql = """
    select   max(signal_id)
    ,        match_latitude
    ,        match_longitude
    ,        max(time_stamp)
    from     signal 
    where    vehicle_id = ? and trip_id = ?
    group by match_latitude, match_longitude
    order by time_stamp
    """

    locations = db.query(sql, (vehicle_id, trip_id))
    return locations

In [10]:
get_trip_locations(*trips[0])

[(16127114, 42.302595, -83.704197, 2500),
 (16127118, 42.302585, -83.704604, 5900),
 (16127121, 42.302574, -83.70513, 9400),
 (16127125, 42.302566, -83.705534, 12500),
 (16127129, 42.302557, -83.705939, 15900),
 (16127132, 42.302548, -83.706375, 19500),
 (16127159, 42.302545, -83.706528, 46500),
 (16127168, 42.302542, -83.706649, 49900),
 (16127174, 42.30254, -83.706729, 53000),
 (16127179, 42.302533, -83.707097, 56200),
 (16127183, 42.302524, -83.707559, 59900),
 (16127188, 42.302513, -83.70813, 62300),
 (16127194, 42.302501, -83.708774, 66300),
 (16127199, 42.302485, -83.709632, 69200),
 (16127203, 42.302463, -83.710274, 71800),
 (16127210, 42.302407, -83.710909, 76300),
 (16127215, 42.30225, -83.711659, 79800),
 (16127220, 42.302085, -83.712138, 82600),
 (16127225, 42.301883, -83.712584, 85300),
 (16127231, 42.301609, -83.713037, 88900),
 (16127234, 42.301155, -83.713604, 90900),
 (16127239, 42.300924, -83.713878, 95900),
 (16127242, 42.300475, -83.714421, 97000),
 (16127245, 42.300

In [6]:
def update_bearing_ini(bearing, vehicle_id, trip_id, time_stamp):
    sql = """update signal set bearing = ? 
             where vehicle_id = ? and trip_id = ? and time_stamp <= ?
          """
    db.execute_sql(sql, [bearing, vehicle_id, trip_id, time_stamp])

In [7]:
def update_bearing_end(bearing, vehicle_id, trip_id, time_stamp):
    sql = """update signal set bearing = ? 
             where vehicle_id = ? and trip_id = ? and time_stamp >= ?
          """
    db.execute_sql(sql, [bearing, vehicle_id, trip_id, time_stamp])

In [8]:
def update_bearing_mid(bearing, vehicle_id, trip_id, ts0, ts1):
    sql = """update signal set bearing = ? 
             where vehicle_id = ? and trip_id = ? and time_stamp > ? and time_stamp <= ?
          """
    db.execute_sql(sql, [bearing, vehicle_id, trip_id, ts0, ts1])

**Warning:** The next cell takes a **long time** to run. On my 2019 i7 MacBook it took a bit over 75 minutes.

In [9]:
for trip in tqdm(trips):
    vehicle_id, trip_id = trip
    
    locations = get_trip_locations(vehicle_id, trip_id)
    
    if len(locations) > 2:   
        lats = [l[1] for l in locations]
        lons = [l[2] for l in locations]

        bearings = vec_bearings(lats, lons)

        l = locations[0]
        update_bearing_ini(bearings[0], vehicle_id, trip_id, locations[0][3])
        update_bearing_end(bearings[-1], vehicle_id, trip_id, locations[-1][3])

        for i in range(1, len(locations)):
            update_bearing_mid(bearings[i-1], vehicle_id, trip_id, locations[i-1][3], locations[i][3])

  0%|          | 0/32552 [00:00<?, ?it/s]